In [9]:
import requests
import json
import pandas as pd
from datetime import *

def fetch(ticker,bc_api_endpoint,apikey,FROM,TO):

    response = requests.get(
                            url=bc_api_endpoint,
                            params={
                                    "tickers":ticker,
                                    "from":FROM,
                                    "to":TO,
                                    },
                                    headers={
                                            "x-api-key":apikey,
                                            },
                            )
    return response

In [10]:
def make_df(ticker_code:str):
    ticker = ticker_code.split(",")
    ####  四半期の財務数値######################################
    bc_api_endpoint = "https://api.buffett-code.com/api/v2/quarter"
    #自分のAPIキーの文字列を挿入
    apikey = "YourApiKey" 

    #期間の指定
    START_q ="2019Q1" #2019年度の第1四半期
    END_q ="2019Q4"   #2019年度の第4四半期


    res_q = fetch(ticker_code,bc_api_endpoint,apikey,START_q,END_q)
    json_data_q = json.loads(res_q.text)
    #バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
    df_q_0 = pd.DataFrame.from_dict(json_data_q[ticker[0]]) #リストの0行目の銘柄
    df_q_1 = pd.DataFrame.from_dict(json_data_q[ticker[1]]) #リストの1行目の銘柄
    df_q_2 = pd.DataFrame.from_dict(json_data_q[ticker[2]]) #リストの2行目の銘柄
    #リスト化をする
    df_q_list = [df_q_0,df_q_1,df_q_2]

    #古い順でデータをソートする
    for i in range(len(df_q_list)):
        #to_datetimeに変換
        df_q_list[i]['to_datetime'] = pd.to_datetime(df_q_list[i]["edinet_updated_date"])
        #ソートしたリストを作成
        sort_data = sorted(df_q_list[i]['to_datetime'])
        #データを再作成
        concat_data = pd.DataFrame()
        for sort_num in range(len(sort_data)):
            #リストを頼りに組み直す
            datetime_data = df_q_list[i][df_q_list[i]['to_datetime'] == sort_data[sort_num]]
            concat_data = pd.concat([concat_data,datetime_data],axis = 0)
        #データを更新
        df_q_list[i] = concat_data

    ##### 日別の株価指標 #########################################
    
    #urlの最後の" / "の後ろをdailyと指定
    bc_api_endpoint = "https://api.buffett-code.com/api/v2/daily"

    year = 2019 #START_dayの一部をyearに代入
    month = 4  #START_dayの一部をmonthに代入
    START_day = f"{year}-0{month}-01" #2019年度の第1四半期の時系列と合わせる
    END_day =  "2020-03-31" #2019年度の第4四半期の時系列と合わせる


    res_day = fetch(ticker_code, bc_api_endpoint, apikey, START_day, END_day) 
    json_data_day = json.loads(res_day.text)    

    #バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
    df_d_0 = pd.DataFrame.from_dict(json_data_day[ticker[0]])
    df_d_1 = pd.DataFrame.from_dict(json_data_day[ticker[1]])
    df_d_2 = pd.DataFrame.from_dict(json_data_day[ticker[2]])
    df_d_list = [df_d_0,df_d_1,df_d_2]
    df_d_list[2]


    #4分割し平均化したデータをリスト化する
    df_day_list = []
    for i in range(len(df_d_list)):
        #datetimeに変換させ時間の比較を行うようにする
        df_d_list[i]['to_datetime'] = pd.to_datetime(df_d_list[i]['day'])

        #mergeのための、空フレームを作成
        df_merge = pd.DataFrame()
        #四半期ごとに分割するために、最初の検索月を指定する際などに利用
        searchConditionMonth = month - 1 
        #四半期ごとに分割するために、+3か月して値を更新する際などに利用
        searchConditionMonthIncrement = 3 
        #yearを代入
        searchConditionYear = year 

        count = 0
        while True:
            count += 1
            #4回分割したら終了(1年分）
            if 4 + 1 <= count:
                break

            # 検索条件（From）～年～月１日より期間スタート
            searchConditionDatetimeFrom = datetime(searchConditionYear, searchConditionMonth + 1, 1, tzinfo=timezone.utc)

            # 検索期間の月と年を更新する
            searchConditionMonth = (searchConditionMonth + searchConditionMonthIncrement) % 12
            # 12で割ったあまりが1(13月 = 10月 + 3月)になった際、年を更新させる
            if searchConditionMonth < searchConditionMonthIncrement:
                searchConditionYear += 1

            # 検索条件（To）～年～(+3)月 で期間をストップ
            searchConditionDatetimeTo = datetime(searchConditionYear, searchConditionMonth + 1, 1, tzinfo=timezone.utc)   

            #検索期間内にあるデータを抽出
            df_dq = df_d_list[i][(searchConditionDatetimeFrom <= df_d_list[i]['to_datetime']) &
                                                                (df_d_list[i]['to_datetime'] < searchConditionDatetimeTo)
                                ]
            #四半期ごとのデータを平均化しmergeさせる
            df_dq_mean = df_dq.mean()
            df_DQ = pd.DataFrame(df_dq_mean)
            df_merge = pd.concat([df_merge,df_DQ],axis = 1)

        #4分割し平均化したデータをリスト化する
        df_day_list.append(df_merge.T) #day
    
    ######  直近営業日の株価指標  ################################################
    bc_api_endpoint = "https://api.buffett-code.com/api/v2/indicator"
    #期間を指定せず、時点(直近営業日)を指定しいているため"None"を記入
    res_c = fetch(ticker_code,bc_api_endpoint,apikey,None,None)
    json_data_c = json.loads(res_c.text)
    #バフェットコードの仕様で一回当たり最大3社が取得可能な為対応
    df_c_0 = pd.DataFrame.from_dict(json_data_c[ticker[0]]) 
    df_c_1 = pd.DataFrame.from_dict(json_data_c[ticker[1]]) 
    df_c_2 = pd.DataFrame.from_dict(json_data_c[ticker[2]]) 
    df_c_list = [df_c_0,df_c_1,df_c_2]

    return df_q_list,df_day_list,df_c_list


In [ ]:
# フォルダを作成
# csvデータ保存する
ticker_code = input()
df_q_list,df_day_list,df_c_list = make_df(ticker_code)
df_q_list[2]

In [6]:
#上記で書いた3種類のデータ取得の関数をもとに集めたデータを、それぞれcsvへ保存
ticker = ticker_code.split(",")

i = 0
for code_num in ticker :
    df_q_list[i].to_csv(f"df_q/{code_num}.csv")
    df_day_list[i].to_csv(f"df_day/{code_num}.csv")
    df_c_list[i].to_csv(f"df_c/{code_num}.csv")
    i += 1 

In [ ]:
with open("ticker_list/ticker.txt" ,mode = "r") as f:
        file = f.read()
        ticker_list_three = file.split("\n")
    
ticker_list_three

In [11]:
try:
    #あらかじめバフェットコード様から抽出できる証券コードのリストを用意(バフェットコード様の業種検索を参考)
    with open("ticker_list/ticker.txt" ,mode = "r") as f:
        file = f.read()
    ticker_list_three = file.split("\n")
    #取得終了時にセーブポイントが上書きされる、初期化する(初めから行いたい）場合はカウントを0にする。
    
    #ticker_listの中に同梱されているticker_count.txtを読み込む(exceptで利用)
    with open("ticker_list/ticker_count.txt" , mode = "r") as f:
        count_int = int(f.read())
    count = count_int
    memory = count
    #カウントを頼りに、ticker_list_threeを上から順番に読み込ませる
    for ticker_code in ticker_list_three[count:] :
        df_q_list,df_day_list,df_c_list = make_df(ticker_code)
        for i in range(3):
            ticker_l = ticker_code.split(",")
            #上記で書いた3種類のデータ取得の関数をもとに集めたデータを、それぞれcsvへ保存
            df_q_list[i].to_csv(f"df_q/{ticker_l[i]}.csv")
            df_day_list[i].to_csv(f"df_day/{ticker_l[i]}.csv")
            df_c_list[i].to_csv(f"df_c/{ticker_l[i]}.csv")
        count += 1
except :
    #リクエスト制限が掛かり一日で取得出来るデータ数が達したらセーブポイントを作成
    with open("ticker_list/ticker_count.txt" , mode = "w") as f:
        f.write(str(count))
    print(f"{count}行で処理が終了しました。")
    print(f"{memory}から始まって{count}まで{count - memory}行読み込まれました。")

0行で処理が終了しました。
0から始まって0まで0行読み込まれました。


In [1]:
industry =  [
             "水産・農林業",
             "鉱業",
             "建設業",
             "食料品",
             "繊維製品",
            "パルプ・紙",
            "化学",
            "医薬品",
            "石油・石炭製品",
            "ゴム製品",
            "ガラス・土石製品",
            "鉄鋼",
            "非鉄金属",
            "金属製品",
            "機械",
            "電気機器",
            "輸送用機器",
            "精密機器",
            "その他製品",
            "電気・ガス業",
            "陸運業",
            "海運業",
            "空運業",
            "倉庫・運輸関連業",
            "卸売業",
            "小売業",
            "銀行業",
            "証券、商品先物取引業",
            "保険業",
            "その他金融業",
            "不動産業",
            "サービス業",
            "情報・通信業",
            "REIT"
            ]

In [2]:
industry_dict={}
for i in industry :
    with open(f"ticker_list/{i}.txt" ,mode = "r") as f:
            file = f.read()
    ticker_list_three = file.split("\n")
    
    industry_list = []
    for code in ticker_list_three :
        ticker_num = code.split(",")
        for count in ticker_num :
            industry_list.append(count)
    
    ind_d = {i:len(industry_list)}
    industry_dict.update(ind_d)
    
print(industry_dict)

{'水産・農林業': 12, '鉱業': 6, '建設業': 158, '食料品': 122, '繊維製品': 54, 'パルプ・紙': 24, '化学': 210, '医薬品': 68, '石油・石炭製品': 12, 'ゴム製品': 20, 'ガラス・土石製品': 56, '鉄鋼': 45, '非鉄金属': 35, '金属製品': 90, '機械': 228, '電気機器': 243, '輸送用機器': 93, '精密機器': 51, 'その他製品': 110, '電気・ガス業': 24, '陸運業': 62, '海運業': 14, '空運業': 6, '倉庫・運輸関連業': 38, '卸売業': 317, '小売業': 348, '銀行業': 83, '証券、商品先物取引業': 41, '保険業': 15, 'その他金融業': 35, '不動産業': 132, 'サービス業': 470, '情報・通信業': 462, 'REIT': 2}


In [6]:
scr_dict = {}
#パラメータ数(指標数)＊10倍以上が機械学習において適正なデータ 今回は12指標なため120以上
scr = 12 * 10
for idk,idv in industry_dict.items():    
    if int(scr) <= idv :
        if_dict = {idk:idv}
        scr_dict.update(if_dict)
        
scr_dict

{'建設業': 158,
 '食料品': 122,
 '化学': 210,
 '機械': 228,
 '電気機器': 243,
 '卸売業': 317,
 '小売業': 348,
 '不動産業': 132,
 'サービス業': 470,
 '情報・通信業': 462}

In [7]:
try:
    industry_select = input("どの業種を取得しますか？ : ")
    with open(f"ticker_list/{industry_select}.txt" ,mode = "r") as f:
            file = f.read()
    ticker_list_three = file.split("\n")
    
    count = 0
    
    for ticker_code in ticker_list_three :
        df_q_list,df_day_list,df_c_list = make_df(ticker_code)
        for i in range(3):
            ticker_l = ticker_code.split(",")
            #上記で書いた3種類のデータ取得の関数をもとに集めたデータを、それぞれcsvへ保存
            df_q_list[i].to_csv(f"df_q/{ticker_l[i]}.csv")
            df_day_list[i].to_csv(f"df_day/{ticker_l[i]}.csv")
            df_c_list[i].to_csv(f"df_c/{ticker_l[i]}.csv")
        
        count += 1
except:
    print(f"全{len(ticker_list_three)}行中{count}行で操作を終了しました。")

どの業種を取得しますか？ : サービス業
全157行中120行で操作を終了しました。
